# Casusopdracht - Films

**Klas:** V2B
**Studenten:** 
- Roan Gaasbeek
- Mathias Hendriks
- Luc Pikaar


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


# 1. Business Understanding

## Doelstelling
Het doel van dit onderzoek is het vaststellen van de belangrijkste kenmerken (zoals budget
IMDb-scores, en sociale media populariteit) van films die goed presteren, om te begrijpen welke
factoren bijdragen aan succes in de filmindustrie.

## Onderzoeksvragen
De volgende onderzoeksvragen worden behandeld in deze opdracht:

1. In hoeverre is de bruto-omzet van een film te voorspellen op basis van de populariteit op **Facebook** en **IMDB**?
2. In hoeverre beïnvloedt het slagen voor de Bechdeltest de bruto-omzet van een film?
3. In hoeverre is het mogelijk om logische clusters te vinden o.b.v. onder andere budget en bruto-omzet? Denk aan blockbusters (hoog budget en hoge omzet), flops (hoog budget en lage omzet) of cultfilms (laag budget en hoge omzet). Je mag extra features toevoegen. 

# 2. Data understanding
De tweede fase in **CRISP-DM** is de **data understanding** dit houdt in:
- Data Collection
- Data Exploration & Analysis

## Data Collection

### Movies dataset
dit is de dataset die we uit deze opdracht hebben meegekregen

### Bechdeltest
De bechdel test is als volgt:
1. It has to have at least two (named) women in it
2. Who talk to each other
3. About something besides a man

https://bechdeltest.com/


In [ ]:
dataset = pd.read_csv('movie.csv')
bechdel_dataset = pd.read_csv('bechdel.csv')

## Data Exploration & Analysis
In deze fase wordt de dataset grondig onderzocht om patronen, relaties en potentieel problematische gegevens te identificeren. Het doel van data exploration is om een eerste inzicht te krijgen in de structuur van de data, trends te ontdekken en belangrijke kenmerken te identificeren die verder onderzocht moeten worden.

### Algemene verkenning
Om een eerste indruk van de dataset te krijgen, maken we gebruik van de `head`-functie. Deze functie geeft een overzicht van de eerste paar rijen van de dataset, waardoor we snel inzicht krijgen in de structuur van de data.

In [ ]:
dataset.head()

Om snel een overzicht te krijgen van alle kolommen en mogelijke features in de dataset, maken we gebruik van de `columns`-functie. Hiermee kunnen we eenvoudig de namen van de kolommen bekijken, wat nuttig is voor het bepalen van welke variabelen relevant zijn voor verdere analyse.

In [ ]:
dataset.columns

Vervolgens gebruiken we de `dtypes`-functie om de datatypen van elke kolom in de dataset te achterhalen. Dit helpt ons te begrijpen welke soort gegevens in elke kolom aanwezig zijn

In [ ]:
dataset.dtypes

We gebruiken de `describe`-functie om een overzicht te krijgen van de belangrijkste statistieken van de numerieke kolommen in de dataset. Deze functie geeft ons inzicht in de gemiddelde waarden, de spreiding, en de extremen van de data, waardoor we beter begrijpen hoe de gegevens zijn verdeeld.

In [ ]:
dataset.describe()

Voor een samenvatting van de niet-numerieke gegevens specificeren we in de `describe`-functie dat we ook objecttypen willen opnemen door `include='object'` toe te voegen.

In [ ]:
dataset.describe(include='object')

### Variabelen & Meetniveaus

In deze analyse zijn de volgende variabelen overwogen met hun verwachte impact op de bruto-omzet (gross):

- Director name & Actor names: De bekendheid van de regisseur en de hoofdrolspelers kan invloed hebben op de aantrekkingskracht van de film en daarmee de bruto-omzet.
- Facebook Likes: De sociale media populariteit van de regisseur en de acteurs kan de zichtbaarheid van de film vergroten en zo invloed op de bruto-omzet hebben.
- Budget: Een hoger budget kan leiden tot hogere bruto-omzet.
- IMDB score: Een hogere score op IMDB wijst vaak op betere filmkwaliteit en kan leiden tot meer kijkers en daarmee hogere bruto-omzet.


| Feature                 | Afhankelijk/Onafhankelijk | Meetniveau |
|:------------------------|:--------------------------|:-----------|
| Director Name           | Onafhankelijk             | Nominaal   |
| Actor 1 Name            | Onafhankelijk             | Nominaal   |
| Actor 2 Name            | Onafhankelijk             | Nominaal   |
| Actor 3 Name            | Onafhankelijk             | Nominaal   |
| Director Facebook Likes | Onafhankelijk             | Discreet   |
| Actor 1 Facebook Likes  | Onafhankelijk             | Discreet   |
| Actor 2 Facebook Likes  | Onafhankelijk             | Discreet   |
| Actor 3 Facebook Likes  | Onafhankelijk             | Discreet   |
| Movie Facebook Likes    | Onafhankelijk             | Discreet   |
| Budget                  | Onafhankelijk             | Continu    |
| IMDB Score              | Onafhankelijk             | Discreet   |
| Gross                   | Afhankelijk               | Continu    |

In [ ]:
kolommen = ['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'director_facebook_likes', 'actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes', 'cast_total_facebook_likes','movie_facebook_likes', 'budget', 'imdb_score', 'gross']
data_analyse = dataset[kolommen]
data_analyse.head()


In [ ]:
data_analyse.describe()

### Target
De targetvariabele in dit onderzoek is de Bruto-omzet `gross`. Deze variabele vertegenwoordigt de inkomsten van een film.

Om inzicht te krijgen in de verspreiding van de bruto-omzetwaarden, zullen er twee histogrammen worden gebruikt. De eerste is zonder limiet hierin is het mogelijk om ook de uitschieters te zien. De tweede is ingezoomd op de waarde tot max 100 Miljoen. Dit helpt om de verdeling van de omzet in de dataset te visualiseren en te identificeren of er sprake is van een normale verdeling, uitschieters of andere patronen die van invloed kunnen zijn op de analyses.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
gross = data_analyse['gross'].dropna()
sns.histplot(gross / 10**6, bins=50, ax=axes[0])
axes[0].set_title('Bruto-omzetverdeling')
axes[0].set_xlabel('Bruto-omzet (Miljoenen)')
axes[0].set_ylabel('Frequentie')

sns.histplot(dataset['gross'] / 10**6, bins=200, ax=axes[1])
axes[1].set_title('Bruto-omzetverdeling met limiet (Max 100 miljoen)')
axes[1].set_xlabel('Bruto-omzet (Miljoenen)')
axes[1].set_ylabel('Frequentie')

# Zet de x-as limiet tot maximaal 200 miljoen
axes[1].set_xlim(0, 100)

plt.tight_layout()
plt.show()

Uit de histogrammen blijkt dat een groot deel van de `gross` waarden zich rond nul bevindt. Dit wijst mogelijk op een aanzienlijk aantal ontbrekende of ongeldige waarden in de dataset. Het kan zijn dat voor veel films de bruto-omzet ontbreekt. Dit zou de betrouwbaarheid van verdere analyses kunnen beïnvloeden.

In [ ]:
omzet = data_analyse['gross'].dropna()
omzet_log = np.log10(omzet + 1)

# Seaborn histogram plotten
plt.figure(figsize=(8, 6))
sns.histplot(omzet_log, bins=50, color='skyblue', edgecolor='black')

# Titel en labels toevoegen
plt.title('Bruto-omzet van films')
plt.xlabel('Bruto-omzet')
plt.ylabel('Aantal films')
plt.xticks(ticks=range(10), labels=["1","10","100","1k","10k","100k","1M", "10M", "100M", "1B"])

plt.grid(True)
plt.show()

Er is gekozen om een log10-transformatie toe te passen op de Bruot-omzet in plaats van een standaard histogram. Dit komt doordat er in de dataset veel films zijn met een bruto-omzet van 0 of zeer lage waarden. Door deze waarden op een lineaire schaal weer te geven, zouden de meeste films in een enkele categorie vallen, wat een "saaie" visualisatie oplevert met weinig inzicht in de spreiding van de hogere bruto-omzetten. De logaritmische schaal comprimeert de hoge uitschieters en spreidt de lagere waarden beter uit, waardoor de onderlinge verschillen beter zichtbaar worden en de visualisatie meer informatieve waarde krijgt.

### Features

#### Facebook likes
Om inzicht te krijgen in de populariteit van de verschillende actoren en de regisseur, hebben we de verdeling van de Facebook-likes per persoon afzonderlijk gevisualiseerd in de vorm van histogrammen. Deze grafieken tonen de frequentie van het aantal likes van de `director`, `acteur 1`, `acteur 2`, `acteur 3`, `cast_total` en de `movie`.

In [ ]:
dataset['director_facebook_likes'].describe()

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(12, 10))

# Logaritmische transformatie van de 'director_facebook_likes' om uitschieters te comprimeren
director_likes = dataset['director_facebook_likes'].dropna()
director_likes_log = np.log10(director_likes + 1)
sns.histplot(director_likes_log, bins=20, kde=False, ax=axes[0, 0])

axes[0, 0].set_title('Regisseur FB Likes')
axes[0, 0].set_xlabel('Aantal Likes')
axes[0, 0].set_ylabel('Frequentie')
axes[0, 0].set_xticks([0, 1, 2, 3, 4, 5])
axes[0, 0].set_xticklabels(["1","10","100","1k","10k","100k"])

# Logaritmische transformatie van de 'actor_1_facebook_likes' om uitschieters te comprimeren
actor_1_likes = dataset['actor_1_facebook_likes'].dropna()
actor_1_likes_log = np.log10(actor_1_likes + 1)
sns.histplot(actor_1_likes_log, bins=20, kde=False, ax=axes[0, 1])

axes[0, 1].set_title('Acteur 1 FB Likes')
axes[0, 1].set_xlabel('Aantal Likes')
axes[0, 1].set_ylabel('Frequentie')
axes[0, 1].set_xticks([0, 1, 2, 3, 4, 5, 6])
axes[0, 1].set_xticklabels(["1","10","100","1k","10k","100k","1M"])

# Logaritmische transformatie van de 'actor_2_facebook_likes' om uitschieters te comprimeren
actor_2_likes = dataset['actor_2_facebook_likes'].dropna()
actor_2_likes_log = np.log10(actor_2_likes + 1)
sns.histplot(actor_2_likes_log, bins=20, kde=False, ax=axes[1, 0])

axes[1, 0].set_title('Acteur 2 FB Likes')
axes[1, 0].set_xlabel('Aantal Likes')
axes[1, 0].set_ylabel('Frequentie')
axes[1, 0].set_xticks([0, 1, 2, 3, 4, 5, 6])
axes[1, 0].set_xticklabels(["1","10","100","1k","10k","100k","1M"])

# Logaritmische transformatie van de 'actor_3_facebook_likes' om uitschieters te comprimeren
actor_3_likes = dataset['actor_3_facebook_likes'].dropna()
actor_3_likes_log = np.log10(actor_3_likes + 1)
sns.histplot(actor_3_likes_log, bins=20, kde=False, ax=axes[1, 1])

axes[1, 1].set_title('Acteur 3 FB Likes')
axes[1, 1].set_xlabel('Aantal Likes')
axes[1, 1].set_ylabel('Frequentie')
axes[1, 1].set_xticks([0, 1, 2, 3, 4, 5, 6])
axes[1, 1].set_xticklabels(["1","10","100","1k","10k","100k","1M"])

# Logaritmische transformatie van de 'cast_total_facebook_likes' om uitschieters te comprimeren
cast_total_likes = dataset['cast_total_facebook_likes'].dropna()
cast_total_likes_log = np.log10(cast_total_likes + 1)
sns.histplot(cast_total_likes_log, bins=20, kde=False, ax=axes[2, 0])

axes[2, 0].set_title('Cast Total FB Likes')
axes[2, 0].set_xlabel('Aantal Likes')
axes[2, 0].set_ylabel('Frequentie')
axes[2, 0].set_xticks([0, 1, 2, 3, 4, 5, 6])
axes[2, 0].set_xticklabels(["1", "10", "100", "1k", "10k", "100k", "1M"])

# Logaritmische transformatie van de 'movie_facebook_likes' om uitschieters te comprimeren
movie_likes = dataset['movie_facebook_likes'].dropna()
movie_likes_log = np.log10(movie_likes + 1)
sns.histplot(movie_likes_log, bins=20, kde=False, ax=axes[2, 1])

axes[2, 1].set_title('Movie Facebook Likes')
axes[2, 1].set_xlabel('Aantal Likes')
axes[2, 1].set_ylabel('Frequentie')
axes[2, 1].set_xticks([0, 1, 2, 3, 4, 5, 6])
axes[2, 1].set_xticklabels(["1", "10", "100", "1k", "10k", "100k", "1M"])

plt.tight_layout()
plt.show()

#### IMDB-Score

In [ ]:
# Veronderstel dat de dataset al geladen is en IMDb-scores bevat
imdb_scores = data_analyse['imdb_score'].dropna()

# Maak een boxplot voor de IMDb-scores
plt.figure(figsize=(8, 6))
sns.boxplot(x=imdb_scores)

# Instellen van de titel en labels
plt.title('IMDB Scores')
plt.ylabel('Cijfer IMDB')

# Weergeven van de plot
plt.grid(True)
plt.show()


#### Budget

In [ ]:
budget = data_analyse['budget'].dropna()
budget_log = np.log10(budget + 1)

# Seaborn histogram plotten
plt.figure(figsize=(8, 6))
sns.histplot(budget_log, bins=50, color='skyblue', edgecolor='black')

# Titel en labels toevoegen
plt.title('Budget')
plt.xlabel('Budget')
plt.ylabel('Aantal films')
plt.xticks(ticks=range(11), labels=["1","10","100","1k","10k","100k","1M", "10M", "100M", "1B", "10B"])

plt.grid(True)
plt.show()

## Data Kwaliteit

### Missende Waarden
Met de functie `isnull` kunnen we controleren hoeveel missende waarden we hebben. Hier voegen we nog een check aan toe om te kijken hoevel waarden 0 zijn

In [ ]:
(data_analyse.isnull() | (data_analyse == 0)).sum()


### Duplicate data
Met de functie `duplicated` checken we of er duplicates in de data zijn. Dit checken we door de waarde van kolommen `movie_title` en `movie_imdb_link` mee te geven aangezien dit eigenlijk de unieke waarden zouden moeten zijn.

In [ ]:
dataset[dataset.duplicated(['movie_title', 'movie_imdb_link'])]

### Uitschieters

In [ ]:
for kolom in ['director_facebook_likes', 'actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes', 'cast_total_facebook_likes', 'movie_facebook_likes', 'budget', 'gross']:

    # Q1, Q3 en IQR bepalen
    q1 = data_analyse[kolom].quantile(0.25)
    q3 = data_analyse[kolom].quantile(0.75)
    iqr = q3 - q1

    # Limieten uitschieters berekenen
    klein_laag = q1 - 1.5 * iqr
    klein_hoog = q3 + 1.5 * iqr
    extreem_laag = q1 - 3 * iqr
    extreem_hoog = q3 + 3 * iqr

    # Kleine uitschieters (1.5 - 3 IQR)
    kleine_uitschieters = data_analyse[kolom][((data_analyse[kolom] < klein_laag) & (data_analyse[kolom] > extreem_laag)) | ((data_analyse[kolom] > klein_hoog) & (data_analyse[kolom] < extreem_hoog))]
    print(f"{kolom.title()} Kleine uitschieters: {kleine_uitschieters.count()}")
 
    # Extreme uitschieters (3+ IQR)
    extreme_uitschieters = data_analyse[kolom][(data_analyse[kolom] < extreem_laag) | (data_analyse[kolom] > extreem_hoog)]
    print(f"{kolom.title()} Extreme uitschieters: {extreme_uitschieters.count()}")

## Waarnemingen

- movie_facebook_likes heeft 930 extreme uitschieters, dat is behoorlijk veel. Zoals eerder benoemd, komt dit waarschijnlijk doordat er veel films zijn zonder facebook pagina, wat geregistreerd wordt als 0 likes.

- imdb_score heeft 0 extreme uitschieters. Dit lijkt ons logisch omdat de imdb score voor een film altijd tussen de 1 en 10 ligt. Er zal dus nooit een waarde van bijvoorbeeld 100 moeten zijn, wat een extreme uitschieter zou zijn.

- De kolommen movie_facebook_likes en imdb_score hebben beide 5043 waarden, maar budget en gross hebben minder waarden (4551 en 4159 waarden).  
We zullen hier een oplossing voor moeten vinden, door bijvoorbeeld de rijen met missende waarden te droppen van de dataset.

- Q1 voor movie_facebook_likes is 0, dit betekent dat ten minste 25% van de films 0 facebook likes hebben. Dit komt waarschijnlijk omdat deze films geen facebook-pagina hebben.

# 3. Data preparation


In [ ]:
#We hebben de link voor de imdb pagina niet nodig maar hierin staat wel de imdb_id die we later kunnen gebruiken om een externe dataset te koppelen.

dataset['movie_imdb_link'] = dataset['movie_imdb_link'].str.slice(start=28, stop=-17)

In [ ]:
#Veranderen van een paar column namen
dataset.rename(columns={'director_name':'director', 'director_facebook_likes':'director_likes', 'actor_1_name':'actor_1', 'actor_1_facebook_likes':'actor_1_likes', 'actor_2_name':'actor_2', 'actor_2_facebook_likes':'actor_2_likes', 'actor_3_name':'actor_3', 'actor_3_facebook_likes':'actor_3_likes', 'cast_total_facebook_likes':'cast_total_likes', 'movie_facebook_likes':'movie_likes', 'movie_title':'title', 'movie_imdb_link':'imdb_id'}, inplace=True)

### Drop NaN

In [ ]:
dataset.dropna(subset=['gross'], inplace=True)

### Drop Duplicates 

In [ ]:
# Controleert het aantal duplicates in de dataset.
totaal_duplicates = dataset.duplicated().sum()
print(f"Aantal duplicates in de dataset: {totaal_duplicates}")

dataset = dataset.drop_duplicates()

In [ ]:
## Valuta naar USD voor gross
landen = dataset['country'].dropna().unique()

## Lijst land naar valuta en valuta naar USD
land_naar_valuta = {
    'USA': 'USD',
    'UK': 'GBP',
    'New Zealand': 'NZD',
    'Canada': 'CAD',
    'Australia': 'AUD',
    'Belgium': 'EUR',
    'Japan': 'JPY',
    'Germany': 'EUR',
    'China': 'CNY',
    'France': 'EUR',
    'Mexico': 'MXN',
    'Spain': 'EUR',
    'Hong Kong': 'HKD',
    'Czech Republic': 'CZK',
    'India': 'INR',
    'South Korea': 'KRW',
    'Peru': 'PEN',
    'Italy': 'EUR',
    'Russia': 'RUB',
    'Denmark': 'DKK',
    'Ireland': 'EUR',
    'South Africa': 'ZAR',
    'Iceland': 'ISK',
    'Switzerland': 'CHF',
    'Romania': 'RON',
    'Netherlands': 'EUR',
    'Hungary': 'HUF',
    'Greece': 'EUR',
    'Sweden': 'SEK',
    'Norway': 'NOK',
    'Taiwan': 'TWD',
    'Cambodia': 'KHR',
    'Thailand': 'THB',
    'Slovakia': 'EUR',
    'Bulgaria': 'BGN',
    'Iran': 'IRR',
    'Poland': 'PLN',
    'Turkey': 'TRY',
    'Nigeria': 'NGN',
    'Brazil': 'BRL',
    'Finland': 'EUR',
    'Bahamas': 'BSD',
    'Argentina': 'ARS',
    'Colombia': 'COP',
    'Israel': 'ILS',
    'Egypt': 'EGP',
    'Indonesia': 'IDR',
    'Pakistan': 'PKR',
    'Slovenia': 'EUR',
    'Dominican Republic': 'DOP',
    'United Arab Emirates': 'AED',
    'Kenya': 'KES',
    'Philippines': 'PHP',
    'Libya': 'LYD',
}

#Koers gebruikt van xe.com (27-09-2024)
valuta_naar_usd = {
    'USD': 1,
    'GBP': 1.33,
    'NZD': 0.63,
    'CAD': 0.74,
    'AUD': 0.68,
    'EUR': 1.11,
    'JPY': 0.0069,
    'CNY': 0.14,
    'MXN': 0.05,
    'HKD': 0.13,
    'CZK': 0.044,
    'INR': 0.012,
    'KRW': 0.000076,
    'PEN': 0.268,
    'RUB': 0.0107,
    'DKK': 0.15,
    'ZAR': 0.058,
    'ISK': 0.0074,
    'CHF': 1.184,
    'RON': 0.224,
    'HUF': 0.0028,
    'SEK': 0.0987,
    'NOK': 0.0948,
    'TWD': 0.0317,
    'KHR': 0.000245,
    'THB': 0.030,
    'BGN': 0.57,
    'IRR': 0.000023738816,
    'PLN': 0.26,
    'TRY': 0.029,
    'NGN': 0.00060493026,
    'BRL': 0.183,
    'BSD': 1,
    'ARS': 0.010,
    'COP': 0.00023999137,
    'ILS': 0.27,
    'EGP': 0.0206,
    'IDR': 0.000066187407,
    'PKR': 0.0036020965,
    'DOP': 0.0166,
    'AED': 0.27,
    'KES': 0.0077,
    'PHP': 0.017,
    'LYD': 0.209,
}


## Valuta naar USD voor gross


In [ ]:
dataset['gross'] = dataset.apply(
    lambda row: row['gross'] * valuta_naar_usd.get(land_naar_valuta.get(row['country'], 'USD'), 1),
    axis=1
)

## Valuta naar USD voor budget



In [ ]:
dataset['budget'] = dataset.apply(
    lambda row: row['budget'] * valuta_naar_usd.get(land_naar_valuta.get(row['country'], 'USD'), 1),
    axis=1
)
dataset[['title', 'country', 'budget', 'gross']].head(10)

In [ ]:
dataset.drop(['color', 'duration', 'aspect_ratio', 'genres', 'num_voted_users', 'facenumber_in_poster', 'plot_keywords', 'num_user_for_reviews', 'num_critic_for_reviews', 'content_rating', 'title_year', 'language'], axis = 1, inplace=True)

dataset = dataset[['title', 'director', 'actor_1', 'actor_2', 'actor_3', 'director_likes', 'actor_1_likes', 'actor_2_likes', 'actor_3_likes', 'cast_total_likes','movie_likes', 'budget', 'imdb_score', 'gross']]


In [ ]:
dataset